<a href="https://colab.research.google.com/github/ingenziart/sokrab/blob/main/problem4ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from sklearn.datasets import load_iris #i used this sklean just to load dataset only .
import pandas as pd

iris=load_iris()


In [ ]:
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df.head()



,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


mathematical delicvation



In [ ]:
X = df.values   # shape (150, 4)
n, d = X.shape

MATHEAMATICAL WAY
you must permor datda
Let:

X = original data matrix (n × d)

μ = mean row vector (1 × d)

X_c = X − μ = centered data

Σ = (1 / (n − 1)) * X_cᵀ X_c

correct covarince
Σ = (1 / (n − 1)) * X_cᵀ X_c

incorect cov if you dont center the data
Σ = (1 / (n − 1)) * X_cᵀ X_c

conclusion
PCA requires centering because uncentered covariance adds a rank-1 term (n / (n − 1)) * μᵀ μ, which biases the eigenvectors so that the first principal component points toward the mean instead of the true direction of maximal variance.






In [ ]:
mu = X.mean(axis=0, keepdims=True)   # 1×4 row vector of feature means
Xc = X - mu                           # centered matrix 150×4
print("Mean vector:", mu)
print("First 5 rows of centered data:\n", Xc[:5])


Mean vector: [[5.84333333 3.05733333 3.758      1.19933333]]
First 5 rows of centered data:
 [[-0.74333333  0.44266667 -2.358      -0.99933333]
 [-0.94333333 -0.05733333 -2.358      -0.99933333]
 [-1.14333333  0.14266667 -2.458      -0.99933333]
 [-1.24333333  0.04266667 -2.258      -0.99933333]
 [-0.84333333  0.54266667 -2.358      -0.99933333]]


this one id for covariance
**bold text**

In [ ]:
Sigma = (Xc.T @ Xc) / (n - 1) #where sigma are Σ
print("Covariance matrix Σ:\n", Sigma)


Covariance matrix Σ:
 [[ 0.68569351 -0.042434    1.27431544  0.51627069]
 [-0.042434    0.18997942 -0.32965638 -0.12163937]
 [ 1.27431544 -0.32965638  3.11627785  1.2956094 ]
 [ 0.51627069 -0.12163937  1.2956094   0.58100626]]


In [ ]:
# (b) eigenvalues/eigenvectors of covariance matrix
import numpy as np

eigvals, eigvecs = np.linalg.eigh(Sigma)  # for symmetric matrices

# sort in descending order of eigenvalues
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]   # columns are eigenvectors (PC directions)

print("Eigenvalues (sorted):", eigvals)
print("Eigenvectors (columns of eigvecs):\n", eigvecs)


Eigenvalues (sorted): [4.22824171 0.24267075 0.0782095  0.02383509]
Eigenvectors (columns of eigvecs):
 [[-0.36138659  0.65658877  0.58202985  0.31548719]
 [ 0.08452251  0.73016143 -0.59791083 -0.3197231 ]
 [-0.85667061 -0.17337266 -0.07623608 -0.47983899]
 [-0.3582892  -0.07548102 -0.54583143  0.75365743]]


***What they mean in PCA***
Each eigenvalue = variance captured along its corresponding principal component.


λ1 is the variance along the 1st principal component (PC1), etc.

Eigenvectors (columns of eigvecs):
Each eigenvector is a direction in the original 4D feature space:
4 entries = combination of [sepal length, sepal width, petal length, petal width].
The first eigenvector (column 0) is PC1, the direction of maximum variance.
They are orthonormal: unit length and mutually perpendicular.

in summary
In PCA, eigenvalues = “how much variance in this direction”,
eigenvectors = “which direction in feature space”.



In [ ]:
# total variance = sum of eigenvalues
total_var = np.sum(eigvals)

# For k = 2
k2 = 2
W2 = eigvecs[:, :k2]          # 4×2 matrix, first two PCs as columns
Z2 = Xc @ W2                  # 150×2 projected data

# variance preserved by first 2 PCs
var_preserved_2 = np.sum(eigvals[:2]) / total_var
print("Variance preserved with k=2:", var_preserved_2)

# For k = 3
k3 = 3
W3 = eigvecs[:, :k3]          # 4×3
Z3 = Xc @ W3                  # 150×3

var_preserved_3 = np.sum(eigvals[:3]) / total_var
print("Variance preserved with k=3:", var_preserved_3)


Variance preserved with k=2: 0.9776852063187947
Variance preserved with k=3: 0.9947878161267244


In [ ]:
# (d) Reconstruction from k=2
Xc_hat_2 = Z2 @ W2.T          # 150×4, reconstructed centered data
X_hat_2 = Xc_hat_2 + mu       # add mean back if you want original scale

# (d) Reconstruction from k=3
Xc_hat_3 = Z3 @ W3.T
X_hat_3 = Xc_hat_3 + mu


In [ ]:
mse_2 = np.mean((Xc - Xc_hat_2)**2)
mse_3 = np.mean((Xc - Xc_hat_3)**2)

print("Reconstruction MSE (k=2):", mse_2)
print("Reconstruction MSE (k=3):", mse_3)


Reconstruction MSE (k=2): 0.02534107393239825
Reconstruction MSE (k=3): 0.005919048088406608


PCA with
k<d reconstructs the data using only the first 𝑘
k principal directions. The remaining components (with smaller eigenvalues) are discarded, which means we lose the variation that lies in those low-variance directions. This loss appears as a non-zero reconstruction error:
$\text{MSE}_k = \frac{1}{nd}\,\|X_c - \hat{X}_c^{(k)}\|_F^2$

For the Iris dataset:

With k = 2, the reconstruction MSE is (your value).

With k = 3, the MSE decreases to (your value).

With k = 4, the MSE is essentially zero, because keeping all components gives a perfect reconstruction of the centered data.


**When Is Reconstruction “Good Enough”? **



In practice, we don’t try to make the reconstruction perfect. Instead, we want to keep the smallest number of components that still capture most of the important structure in the data.

A common guideline is:

✔ choose the smallest
𝑘
k such that
explained variance ≥ 95%
and
reconstruction error is small.

In our case:

With k = 2, PCA keeps about (your variance %) of the total variance (usually about 95%+ for Iris).
Even though some information is lost, the 2-D representation still keeps the structure of the data and separates the classes well.

With k = 3, almost all the variance is preserved and the reconstruction error becomes very small.

Adding more than 3 components only reduces the MSE slightly but increases the dimensionality again, so it’s not worth it.



Conclusion


k = 2 is a great balance between compression and interpretability—perfect for visualization.


k = 3 is almost lossless and gives a very close reconstruction of the original centered data.


Beyond 3 components, the improvement is tiny, so reconstruction is already “good enough.”


Just plug in your numbers for:


variance explained with k=2 and k=3


MSE for k=2 and k=3


and this becomes your final report write-up.
	​
